In [1]:
import pandas as pd
import numpy as np
import random

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

In [2]:
avisos = pd.read_csv('Predecir/avisos_detalles_train.csv')
postulantes = pd.read_csv('Predecir/postulantes_datos_train.csv')
postulaciones = pd.read_csv('Predecir/df_postulaciones.csv')

In [3]:
del postulantes['Unnamed: 0']
del avisos['Unnamed: 0']
del postulaciones['Unnamed: 0']

In [4]:
avisos_si = avisos.copy()
avisos_si['se_postulo'] = 1

In [5]:
postulaciones = postulaciones.sample(frac=0.20)

In [6]:
postulaciones = pd.merge(postulaciones, avisos_si, how='inner', on=['idaviso'])

In [7]:
postulaciones = pd.merge(postulaciones, postulantes, how='inner', on=['idpostulante'])

In [8]:
def random_fill(row):
    return np.random.choice(avisos['idaviso'])

In [9]:
while (len(postulaciones[postulaciones["se_postulo"]==1])*0.5 > len(postulaciones[postulaciones["se_postulo"]==0])):
    relleno = postulantes.sample(frac=0.5).copy()
    relleno['idaviso'] = np.nan
    relleno['idaviso'] = relleno.apply(random_fill, axis=1)
    relleno = pd.merge(relleno, avisos, how='inner', on=['idaviso'])
    postulaciones = pd.merge(postulaciones, relleno, how='outer')
    postulaciones = postulaciones.fillna({'se_postulo' : 0})

In [10]:
postulaciones.shape

(2052516, 46)

In [11]:
postulaciones = postulaciones.dropna()

In [12]:
se_postulo = postulaciones['se_postulo']
del postulaciones['se_postulo']
del postulaciones['idaviso']
del postulaciones['idpostulante']
postulaciones.head()

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,incorp_inmediata,...,total_vistas,vistas_median,cant_dias,nombre,estado,sexo,edad,vistas_postulante,cant_dias_actividad,postu_postulante
4,1,1,84,293,0,0,0,0,0,0,...,2638.0,2.0,34.0,4.0,1.0,1,26.0,62.0,89.0,62.0
6,1,2,84,107,0,0,1,0,0,0,...,4623.0,2.0,7.0,4.0,1.0,1,26.0,62.0,89.0,62.0
7,1,2,84,3356,0,0,0,0,0,0,...,634.0,1.0,1.0,4.0,1.0,1,26.0,62.0,89.0,62.0
8,1,1,84,2423,0,0,0,0,0,0,...,1137.0,3.0,41.0,4.0,1.0,1,26.0,62.0,89.0,62.0
9,1,2,84,2049,0,0,0,0,0,0,...,384.0,1.0,39.0,4.0,1.0,1,26.0,62.0,89.0,62.0


In [13]:
X_train, X_test, y_train, y_test = train_test_split(postulaciones, se_postulo, test_size=0.15, random_state=7)

In [14]:
seed = 7
num_trees = 10
kfold = model_selection.KFold(n_splits=5, random_state=seed)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold)
print(results.mean())

0.8380462298736984


In [15]:
model.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=10, random_state=7)

In [16]:
to_predict  = pd.read_csv('Predecir/test_final_100k.csv')
to_predict  = pd.merge(to_predict, avisos, on = "idaviso",how="left")
to_predict = pd.merge(to_predict, postulantes, on = "idpostulante",how="left")
to_predict = to_predict.drop_duplicates(["idaviso","idpostulante"])

In [17]:
to_predict.shape

(100000, 46)

In [18]:
del to_predict["idaviso"]
del to_predict["idpostulante"]
to_predict = to_predict.dropna()

In [19]:
to_sumit = model.predict(to_predict.drop("id",axis = "columns"))

In [20]:
to_predict["prediccion"] = to_sumit

In [21]:
res  =  pd.read_csv('Predecir/test_final_100k.csv')

res = pd.merge(res, avisos, on = "idaviso",how="left")
res = pd.merge(res, postulantes, on = "idpostulante",how="left")
res = res.drop_duplicates(["idaviso","idpostulante"])

In [22]:
to_sumit = pd.merge(res,to_predict,on="id",how="left")["prediccion"]
to_sumit = to_sumit.fillna(random.getrandbits(1))
df = pd.DataFrame()
df["sepostulo"] = to_sumit
df["id"] = pd.read_csv('Predecir/test_final_100k.csv')["id"]
final = df.set_index("id")
final.shape

(100000, 1)

In [23]:
final.to_csv("final.csv")